# * Ex-T Sales

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Ex-T Sales
->  DIM_CORP_KPI

In [5]:
''' Execute query '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Ex-T Revenue
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Ex-T Sales DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-06, 19:15:08

   -> Ex-T Sales DataFrame : 829 rows, 17 columns

TDMDBPR : Disconnected


In [6]:
''' METRIC Summary '''

src_t_sales_df = chk_src_df.copy()
src_t_sales_df = src_t_sales_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sales_df = src_t_sales_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_t_sales_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_sales_df[col] = src_t_sales_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_sales_df.to_string(max_cols=10)}')


     PRODUCT       METRIC_CD                                                             METRIC_NAME           LOAD_DATE  START_DAY   END_DAY              P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   POSTPAID    DB2R010500CG                             Postpaid Inflow M1 B2C : DTAC - GEO Channel 2025-03-06 14:17:47   20240101  20250303    148,951,159              7          798   209054
1   POSTPAID    DB2R010500CS                                           Postpaid Inflow M1 B2C : DTAC 2025-03-06 14:17:47   20240101  20250303    383,190,250              7         6204   660115
2   POSTPAID    DB2R020500CG                             Postpaid Inflow M1 B2B : DTAC - GEO Channel 2025-03-06 14:17:47   20240101  20250303      6,018,790              7          311    22880
3   POSTPAID    DB2R020500CS                                           Postpaid Inflow M1 B2B : DTAC 2025-03-06 14:17:47   20240101  20250303    139,028,551              7         1528    75027
4   POSTPAID    DB2S010100CG 

In [7]:
''' METRIC by Month '''

v_tm_key_mth = '202502'
src_t_sales_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

src_t_sales_df = chk_src_df[src_t_sales_df_cols].copy()
src_t_sales_df = src_t_sales_df.loc[src_t_sales_df['TM_KEY_MTH']==v_tm_key_mth]
src_t_sales_df = src_t_sales_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_sales_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_sales_df[col] = src_t_sales_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_sales_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202502

     PRODUCT       METRIC_CD                                                             METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C            P            G            H           HH         CCAA       CCAATT
0   POSTPAID    DB2R010500CG                             Postpaid Inflow M1 B2C : DTAC - GEO Channel 2025-03-06 14:17:47     202502   20250201  20250228  0    7,007,921    6,884,284    6,884,284    6,884,284    6,884,284    6,884,284
1   POSTPAID    DB2R010500CS                                           Postpaid Inflow M1 B2C : DTAC 2025-03-06 14:17:47     202502   20250201  20250228  0   17,944,448   17,677,112   17,677,112   17,677,112   17,677,112   17,677,112
2   POSTPAID    DB2R020500CG                             Postpaid Inflow M1 B2B : DTAC - GEO Channel 2025-03-06 14:17:47     202502   20250201  20250228  0      382,012      360,044      360,044      360,044      360,044      360,044
3   POSTPAID   